## Snowflake Feedback Functions
This example notebook computes feedback functions in deferred mode inside snowflake warehouse. 
Trulens library imported in the LLM app just takes care of logging to snowflake.

In [ ]:
from trulens.connectors.snowflake import SnowflakeConnector
from trulens.core.schema.app import RecordIngestMode
from trulens.core.session import TruSession

connection_params = {}

connector = SnowflakeConnector(**connection_params)
session = TruSession(connector=connector)

In [ ]:
from snowflake.snowpark import Session

snowpark_session = Session.builder.configs(connection_params).create()

In [ ]:
from typing import List

from snowflake.core import Root


class CortexSearchRetriever:
    def __init__(self, session: Session, limit_to_retrieve: int = 4):
        self._session = session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._session)
        cortex_search_service = (
            root.databases["SERVERSIDE_DEMO"]
            .schemas["DATA"]
            .cortex_search_services["JOSH_CORTEX_SEARCH_SERVICE"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["doc_text"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["doc_text"] for curr in resp.results]
        else:
            return []

In [ ]:
from snowflake.cortex import Complete
from trulens_eval.tru_custom_app import instrument


class RAG_from_scratch:
    def __init__(self):
        self.retriever = CortexSearchRetriever(
            session=session, limit_to_retrieve=4
        )

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""
          You are an expert assistant extracting information from context provided.
          Answer the question based on the context. Be concise and do not hallucinate.
          If you don´t have the information just say so.
          Context: {context_str}
          Question:
          {query}
          Answer:
        """
        return Complete("mistral-large", prompt)

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG_from_scratch()

In [ ]:
import numpy as np
from trulens.core import Select
from trulens.core.feedback.feedback import SnowflakeFeedback
from trulens.providers.cortex import Cortex

provider = Cortex(
    connection_params,
    model_engine="mistral-large2",
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    SnowflakeFeedback(
        provider.relevance_with_cot_reasons, name="Answer Relevance"
    )
    .on_input()
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    SnowflakeFeedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="RAG",
    app_version="v1",
    feedbacks=[
        f_answer_relevance,
        # f_context_relevance,
    ],
    record_ingest_mode=RecordIngestMode.BUFFERED,
)

In [ ]:
with tru_rag as recording:
    for i in range(10):
        resp = rag.query(
            f"Question {i} When is University of Washington founded ?"
        )
        print(resp)